In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from keras import layers, Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset.info()

In [ ]:
dataset['sex'].value_counts(normalize=True)

In [ ]:
dataset['children'].value_counts()

In [ ]:
dataset['smoker'].value_counts(normalize=True)

In [ ]:
dataset['region'].value_counts(normalize=True)

In [ ]:
def processing(input):
  data = input.copy()
  data['sex'] = data['sex'].astype('category')
  data['smoker'] = data['smoker'].astype('category')
  data['region'] = data['region'].astype('category')
  data = pd.get_dummies(data, drop_first=True, columns=['sex', 'smoker', 'region'])
  return data

processed_data = processing(dataset)
processed_data.head()

In [ ]:
train_dataset = processed_data.sample(frac=0.8)
test_dataset = processed_data.iloc[~(processed_data.index.isin(train_dataset.index))]
train_dataset.shape, test_dataset.shape

In [ ]:
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

In [ ]:
# numeric = ('age', 'bmi', 'children')
# categorical = ('sex', 'smoker', 'region')
# features = []
# for name in categorical:
#   unique_val = processed_data[name].unique()
#   features.append(
#       tf.feature_column.categorical_column_with_vocabulary_list(name, unique_val)
#   )
# for name in numeric:
#   features.append(
#       tf.feature_column.numeric_column(name)
#   )
# features

In [ ]:
model = Sequential()
# model.add(layers.Dense(10, activation='linear'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(4, activation='linear'))
# model.add(layers.Dense(4, activation='relu'))
model.add(layers.Dense(2, activation='linear'))
# model.add(layers.Dense(2, activation='relu'))
model.add(layers.Dense(1, activation='linear'))
# model.add(layers.Dense(1, activation='relu'))
model.compile(optimizer=Adam(learning_rate=0.05), loss='mae', metrics=['mae','mse'])
# model.summary()

In [ ]:
es = EarlyStopping(monitor='loss', min_delta=0.001, patience=5, verbose=1)
history = model.fit(train_dataset, train_labels, batch_size=32, epochs=40, verbose = 1, callbacks=[es])

In [ ]:
# model.layers[-1].get_weights()
# model.weights

In [ ]:
model.evaluate(train_dataset, train_labels)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
